In [1]:
import pandas as pd
import os
from tqdm import tqdm
import numpy as np
from transformers import GPT2Tokenizer, GPT2LMHeadModel

C:\Users\GLw\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv("data/final_data.csv")

In [3]:
#def add_special_tokens(df):
    # Concatenate the prompts and texts with a special token in between
#    df['combined_text'] = "<|original|> Original text: "+ df['original_text'] + " <|rewritten|> Rewritten text:" + df['rewritten_text'] + "<|prompt|>" + " The prompt that instructing language model to rewrite original text into rewritten text is: " + df['prompt']
#    df['combined_text_inf'] = "<|original|> Original text: " + df['original_text'] + " <|endoftext|> Rewritten text: " + df['rewritten_text'] + "<|prompt|>" + " The prompt that instructing language model to rewrite original text into rewritten text is: "
#    return df

def add_special_tokens(df):
    # Concatenate the prompts and texts with a special token in between
    df['combined_text'] = "Original text: "+ df['original_text'] + " <|endoftext|> Rewritten text:" + df['rewritten_text'] + "<|endoftext|> The prompt used to generate rewritten text given original text is: " + df['prompt']
    df['combined_text_inf'] = "Original text: " + df['original_text'] + " <|endoftext|> Rewritten text: " + df['rewritten_text'] + "<|endoftext|> The prompt used to generate rewritten text given original text is: "
    return df

# Apply the function to each row in the 'rewritten_text' column
df = add_special_tokens(df)
df.head()

,original_text,prompt,rewritten_text,combined_text,combined_text_inf
0,"Hollis Price (born October 29, 1979) is an Ame...","Convert the text into a recipe, where each ins...",Hollis Price's Basketball DreamIngredients:- 1...,"Original text: Hollis Price (born October 29, ...","Original text: Hollis Price (born October 29, ..."
1,Grangewoood Independent School is an independe...,Transform the text into a cryptic crossword pu...,Across1. A school situated in a London borough...,Original text: Grangewoood Independent School ...,Original text: Grangewoood Independent School ...
2,Tap... tap... tap. I could hear him hiding som...,Translate the essence of this text into a horr...,"A tap, tap, tap echoed through the walls, send...",Original text: Tap... tap... tap. I could hear...,Original text: Tap... tap... tap. I could hear...
3,A man smiled as he saw a package underneath hi...,Write like Oscar Wilde: Adopt Wilde's witty an...,"With a groan, he hoisted the box, an embodimen...",Original text: A man smiled as he saw a packag...,Original text: A man smiled as he saw a packag...
4,After weeks of making phone calls we finally f...,Rewrite this text in the style of a time trave...,Weeks of frantic phone calls yielded a celesti...,Original text: After weeks of making phone cal...,Original text: After weeks of making phone cal...


In [4]:
import gc
import torch
gc.collect()
torch.cuda.empty_cache() 

from sentence_transformers import SentenceTransformer

scs_model = SentenceTransformer("sentence-t5-base")

def  batch_cosine_similarity(x1, x2):

    # Normalize the embeddings to unit vectors
    x1_norm = torch.nn.functional.normalize(x1, p=2, dim=-1)
    x2_norm = torch.nn.functional.normalize(x2, p=2, dim=-1)
    
    # Compute the cosine similarity
    cos_sim = torch.mm(x1_norm, x2_norm.transpose(0, 1))
    
    return cos_sim

def sharpened_cosine_similarity_batch(scs_model, output_texts, target_texts, sharpen_factor=3):
    # Assuming scs_model.encode() returns PyTorch tensors
    target_embeddings = scs_model.encode(target_texts, convert_to_tensor=True)
    output_embeddings = scs_model.encode(output_texts, convert_to_tensor=True)
    
    # Calculate batch cosine similarities using the previously defined batch_cosine_similarity function
    cos_sims = batch_cosine_similarity(target_embeddings, output_embeddings)
    
    # Apply the sharpening factor to each similarity score and create a list of tensors
    sharpened_scores = [cos_sims[i][i].unsqueeze(0) ** sharpen_factor for i in range(cos_sims.size(0))]
    
    return sharpened_scores


In [5]:
tokenizer = GPT2Tokenizer.from_pretrained('model/gpt2/')
model = GPT2LMHeadModel.from_pretrained('model/gpt2/')

In [6]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50258, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50258, bias=False)
)

In [8]:
df.shape

(8673, 5)

In [10]:
scs = []
CLEAN=0
for i in tqdm(range(8100,8673)):#input_text = "I get it now. You have to give to get. Where's did I put the cereal? You ca n't have life without death. Or love without hate. I think this milk is sour. Maybe it's fine. Maybe I'm fine, just the way I am.   I have to shower before I go. Maybe the only reason everything is so shit is because I expect it to be. A positive attitude might be the only thing I need. Shit, I'm going to be late.   Learn to drive, asshole! I need to relax, like in that dream, some real Zen stuff. Love and equality and all that. Fuck, I spilled my coffee. Today is going to be hell.<|endoftext|> The cosmic tapestry of existence unraveled before me, illuminating the interconnectedness of all phenomena. I perceive the enigmatic axiom woven through the threads of life and death, love and hate. It is an axiom inscribed in the cosmic consciousness, a testament to the duality of existence.Through the prism of perception, I discern the paradoxical dance of giving and receiving. To acquire the sustenance necessary for survival, one must offer a reciprocal gift, a harmonious exchange of energy. The cosmic rhythm dictates that to emerge unscathed, one must embrace the yin and yang of life, death, and all that encompasses them.The sourness of the milk serves as a reminder of the impermanence of all temporal endeavors. Yet, within the abyss of doubt, I find solace in the recognition of my own intrinsic wholeness. Perhaps, the paradox of existence is reconciled through a profound sense of self-<|endoftext|>"  # This is your input text
    input_text = df["combined_text_inf"][i]
    prompt = df["prompt"][i]
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)

    # Generate text using the model
    # Adjust the max_length parameter as needed
    output_sequences = model.generate(
        input_ids=input_ids,
        max_length=1024,  # Adjust the max length of the output text if needed
        temperature=0.1,  # Temperature controls the randomness of the output
        top_k=50,  # Top-k sampling
        top_p=0.95,  # Nucleus sampling
        repetition_penalty=1.2,
        do_sample=True,
        num_return_sequences=1  # Number of output sequences to generate
    )

    # Decode the output
    generated_text = tokenizer.decode(output_sequences[0], skip_special_tokens=False)

    #print("Generated text:", generated_text)
    pred_prompt = generated_text.split("The prompt used to generate rewritten text given orignal text is:")[-1]
    a=[pred_prompt]
    b=[prompt]
    score = sharpened_cosine_similarity_batch(scs_model, a, b, sharpen_factor=3)
    scs.append(score[0].item())
    CLEAN+=1
    if CLEAN%100==0:
        gc.collect()
        torch.cuda.empty_cache() 



  0%|          | 0/573 [00:00<?, ?it/s]


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
np.mean(scs)

0.4351041913032532

In [19]:
i=8003

input_text = df["combined_text_inf"][i]
prompt = df["prompt"][i]
input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)

    # Generate text using the model
    # Adjust the max_length parameter as needed
output_sequences = model.generate(
    input_ids=input_ids,
    max_length=len(input_ids[0])+40,  # Adjust the max length of the output text if needed
    temperature=1.0,  # Temperature controls the randomness of the output
    top_k=50,  # Top-k sampling
    top_p=0.95,  # Nucleus sampling
    repetition_penalty=1.2,
    do_sample=True,
    num_return_sequences=1  # Number of output sequences to generate
)

    # Decode the output
generated_text = tokenizer.decode(output_sequences[0], skip_special_tokens=False)

    #print("Generated text:", generated_text)
pred_prompt = generated_text.split("The prompt used to generate rewritten text given original text is:")[-1]


a=[pred_prompt]
b=[prompt]
score = sharpened_cosine_similarity_batch(scs_model, a, b, sharpen_factor=3)
print(score)
print(pred_prompt)
print(prompt)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[tensor([0.3568], device='cuda:0')]
  """"""" Dear Diary Hero,"" I wrote yesterday morning through my worn laptop screen while browsing news archives at /r/fallenworldnews. As usual it took me quite some time even writing
Turn it into a classic Greek tragedy.


In [ ]:
generated_text

"Original text: Karanpura is an archeological site near Bhadra city of Hanumangarh district in Rajasthan, India. It belongs with ancient Indus Valley Civilization. Harappan pottery has been found after excavation. <|endoftext|> Rewritten text: Karanpura 🏭️ is an archeological site near Bhadra 🌃 city of Hanumangarh 🗺️ district in Rajasthan 🇮🇳, India. It belongs with ancient Indus Valley Civilization 🌍. Harappan urns pottery has been found after excavation.<|endoftext|> The prompt used to generate rewritten text given original text is:  `` Dear reader ’ s detective ( Mr Miller ), we've unearthed a cache that contains the finest preserved Roman and Medieval coins from around Ancient Rome! Excavation revealed a wealth trove depicting treasures such"

In [ ]:
generated_text.split("The prompt used to generate rewritten text given orignal text is:")[-1]

' 2016 Presidential Race Script"\n"Original text: He was dead. He went down. He had massive trauma to his face. If he was n\'t dead, he would be'

In [ ]:
tt = "<|endoftext|> Rewritten text:This is a very inspiring article I have a question about your rmse"
tt

'<|endoftext|> Rewritten text:This is a very inspiring article I have a question about your rmse'

In [ ]:
a=["1920s Gangster Slang"]
b=[prompt]
sharpened_cosine_similarity_batch(scs_model, a, b, sharpen_factor=3)

[tensor([0.3861], device='cuda:0')]

In [ ]:
print(1) if 1==2 else print(2)


2
